In [4]:
import time 
import cv2 as cv 
import mediapipe as mp
import numpy as np
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [5]:
cap=cv.VideoCapture(0)
mpHands=mp.solutions.hands
hands=mpHands.Hands(min_detection_confidence=0.7)
mpDraw=mp.solutions.drawing_utils
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
ra=volume.GetVolumeRange()
minv=ra[0]
maxv=ra[1]
vr=400
vp=ptime=0

while 1:
    _,img=cap.read()
    img=cv.flip(img,1)
    rgb=img[:,:,::-1]
    results=hands.process(rgb)
    l=[]
    if results.multi_hand_landmarks:
        for hand in results.multi_hand_landmarks:
            for iD,lm in enumerate(hand.landmark):
                h,w,c=img.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                cv.circle(img,(cx,cy),1,(0,255,255),-1)
                l.append([iD,cx,cy])
            mpDraw.draw_landmarks(img,hand,mpHands.HAND_CONNECTIONS)
    if len(l)!=0:
        #print(l[4],l[8])
        x1,y1=l[4][1],l[4][2]
        x2,y2=l[8][1],l[8][2]
        xc,yc=(x1+x2)//2,(y1+y2)//2
        cv.circle(img,(x1,y1),15,(155,255,255),-1)
        cv.circle(img,(x2,y2),15,(0,255,255),-1)
        cv.line(img,(x1,y1),(x2,y2),(255,0,255),3)
        cv.circle(img,(xc,yc),15,(255,0,255),-1)
        length=math.hypot(x2-x1,y2-y1)
        #print(length)
        vol=np.interp(length,[25,250],[minv,maxv])
        vr=np.interp(length,[25,250],[400,150])
        vp=np.interp(length,[25,250],[0,100])
        cv.putText(img,f'{int(vp)}%',(40,450),cv.FONT_HERSHEY_COMPLEX,1,(0,255,0),1)
        volume.SetMasterVolumeLevel(vol, None)
        cv.rectangle(img,(50,150),(85,400),(0,255,0),3)
        cv.rectangle(img,(50,int(vr)),(85,400),(0,255,0),-1)        
        if length<30:
            cv.circle(img,(xc,yc),15,(0,0,255),-1)
    ctime=time.time()
    fps=1/(ctime-ptime)
    ptime=ctime
    cv.putText(img,str(int(fps)),(10,30),cv.FONT_HERSHEY_COMPLEX,1,(255,0,0),1)
    cv.imshow('img',img)
    if cv.waitKey(1)==ord('q'):
        break
cap.release()
cv.destroyAllWindows()
    

In [3]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
ra=volume.GetVolumeRange()
minv=ra[0]
maxv=ra[1]
volume.SetMasterVolumeLevel(-60.0, None)

0